In [ ]:
import pythreejs as p3
import numpy as np
import ipywidgets as w
import matplotlib.cm as cm

In [ ]:
scalar_map = cm.ScalarMappable(cmap='viridis')

In [ ]:
N = 100
#c = np.random.random([N**3, 4])
#pos = 10.0*(np.random.random([N, 3]) - 0.5)
p = np.linspace(-5.0, 5.0, N)
x, y, z = np.meshgrid(p, p, p)
r = np.sqrt(x.ravel()**2 + y.ravel()**2 + z.ravel()**2)
a = np.sin(r)
c = scalar_map.to_rgba(a).astype(np.float32)
c[:, 3] = 0.1

In [ ]:
pos = np.array([x.ravel(), y.ravel(), z.ravel()]).T
geometry = p3.BufferGeometry(
    attributes={
        'position':
        p3.BufferAttribute(array=pos),
        'rgba_color':
        p3.BufferAttribute(array=c),
        'color':
        p3.BufferAttribute(array=np.random.random([N, 3])),
#         'size': 20.
    })
material = p3.ShaderMaterial(
    vertexShader='''
precision highp float;
attribute vec4 rgba_color;
varying vec3 mypos;
varying vec4 vColor;
varying vec4 projected;
float xDelta, yDelta, zDelta, delta;
void main(){
    vColor = rgba_color;
//    vColor = vec4(color, 1.0);
    projected = projectionMatrix * modelViewMatrix * vec4(position, 1.0);
    gl_Position = projected;
    // use the delta between the point position and camera position to size point
    xDelta = pow(position[0] - cameraPosition[0], 2.0);
    yDelta = pow(position[1] - cameraPosition[1], 2.0);
    zDelta = pow(position[2] - cameraPosition[2], 2.0);
    delta = pow(xDelta + yDelta + zDelta, 0.5);
    gl_PointSize = 40.0 / delta;
}
''',
    fragmentShader='''
precision highp float;
varying vec4 vColor;
void main() {
    gl_FragColor = vColor;
}
''',
    vertexColors='VertexColors',
    transparent=True,
    depthTest=False,
    alphaTest=0.5,
#     polygonOffset = True,
#     polygonOffsetFactor = -4
)
# material = p3.PointsMaterial(vertexColors='VertexColors',
# #                                          size=self._pixel_size,
#                                          transparent=True)
points = p3.Points(geometry=geometry,
                        material=material)

In [ ]:
view_width = 600
view_height = 400
camera = p3.PerspectiveCamera(position=[20, 0, 0], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
scene = p3.Scene(children=[points, camera, key_light, ambient_light], background="#ffffff")
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer

In [ ]:
def update_slice_pos(change):
#    newc = np.where(np.abs(pos[:, 0] - change["new"]) < 0.5, 1.0, 0.1)
#     newc = np.where(np.abs(np.sqrt(
#         pos[:, 0]*pos[:, 0] + pos[:, 1]*pos[:, 1] + pos[:, 2]*pos[:, 2]) - change["new"]) < 0.5, 1.0, 0.0)
    newc = np.where(np.abs(np.sqrt(
        pos[:, 0]*pos[:, 0] + pos[:, 1]*pos[:, 1]) - change["new"]) < 0.5, 1.0, 0.0)
    c3 = geometry.attributes["rgba_color"].array
    c3[:, 3] = newc
    geometry.attributes["rgba_color"].array = c3
#sl = w.FloatSlider(min=-5, max=5, value=0)
sl = w.FloatSlider(min=0, max=8, value=0)
sl.observe(update_slice_pos, names="value")
sl